### Análisis exploratorio de los datos

* Análisis datos.
    * Promedio/desvío tweets por período (bloque, semana, día) por municipio/global.
    * Municipios con tweets superiores al promedio en cada unidad.
        * Seleccionar aquellos que tengan más que el promedio para cada bloque ?
            * Deja pocos usuarios...

In [ ]:
import pandas as pd

In [ ]:
data_dir = './df_merged/'

df_tweets_db = pd.read_pickle(data_dir + 'df_tweets_db_processed.pickle')
len(df_tweets_db)

In [ ]:
df_tweets_db['hashtag_count'] = df_tweets_db['hashtags'].map(len)
df_tweets_db['media_count'] = df_tweets_db['media'].map(len)
df_tweets_db['mentions_count'] = df_tweets_db['mentions'].map(len)
df_tweets_db['url_count'] = df_tweets_db['url'].map(len)

In [ ]:
df_users_db = pd.read_pickle(data_dir + 'df_users_db.pickle')
len(df_users_db) # este no está filtrado, pero los tweets si

#### Filtrado de muninicipios que no tuvieron tweets pre, durante y post-covid

Filtramos los tweets de acuerdo a los límites, agrupamos y calculamos las cantidades para luego ver los user_ids que no tengan en todos los períodos.

Guardar las stats y el set de tweets obtenido en un nuevo df.

In [ ]:
from datetime import datetime

no_covid_2019 = datetime(2019,1,1)
no_covid = datetime(2020,1,1)
pre_covid = datetime(2020,3,1)
post_covid = datetime(2022,1,1)

def get_stats(df,no_covid, pre_covid,post_covid,min_date=None,column='user_id', agrr='count'):
    if min_date is not None:
        df_stats = df[df['created_at'] >= pd.to_datetime(min_date)]
    
    no_covid = pd.to_datetime(no_covid)
    pre_covid = pd.to_datetime(pre_covid)
    post_covid = pd.to_datetime(post_covid)
    
    df_stats = df[df['created_at'] <= no_covid].groupby(by=[column])[[column]].agg(agrr)
    df_stats = df_stats.rename(columns={column:'no_covid'})
    df_stats = pd.concat([df_stats, df[(df['created_at'] > no_covid) & (df_tweets_db['created_at'] <= pre_covid)].groupby(by=[column])[[column]].agg(agrr).rename(columns={column:'pre_covid'})], axis=1)
    df_stats = pd.concat([df_stats, df[(df['created_at'] <= post_covid) & (df_tweets_db['created_at'] > pre_covid)].groupby(by=[column])[[column]].agg(agrr).rename(columns={column:'during_covid'})], axis=1)
    df_stats = pd.concat([df_stats, df[df['created_at'] > post_covid].groupby(by=[column])[[column]].agg(agrr).rename(columns={column:'post_covid'})], axis=1)
    df_stats = df_stats.fillna(0)
    return df_stats

In [ ]:
df_stats = get_stats(df_tweets_db,no_covid, pre_covid,post_covid,min_date=no_covid_2019,agrr='count')
df_stats.to_pickle('df_date_stats.pickle')
df_stats

# df_tweets_db[df_tweets_db['created_at'] < pd.to_datetime(no_covid_2019)]

In [ ]:
len(df_stats[df_stats['no_covid'] > 0]), len(df_stats[df_stats['pre_covid'] > 0]), len(df_stats[df_stats['during_covid'] > 0]), len(df_stats[df_stats['post_covid'] > 0])

In [ ]:
# solo 87 cuentas tienen tweets en todos los períodos
df_stats = df_stats[(df_stats['no_covid'] > 0) & (df_stats['pre_covid'] > 0) & (df_stats['during_covid'] > 0) & (df_stats['post_covid'] > 0)]

import numpy as np

perc = 0
print(perc, len(df_stats[(df_stats['pre_covid'] >= np.percentile(df_stats['pre_covid'].values,perc)) & 
         (df_stats['during_covid'] >= np.percentile(df_stats['during_covid'].values,perc)) & 
         (df_stats['post_covid'] >= np.percentile(df_stats['post_covid'].values,perc))]))

perc = 5
print(perc, len(df_stats[(df_stats['pre_covid'] >= np.percentile(df_stats['pre_covid'].values,perc)) & 
         (df_stats['during_covid'] >= np.percentile(df_stats['during_covid'].values,perc)) & 
         (df_stats['post_covid'] >= np.percentile(df_stats['post_covid'].values,perc))]))


perc = 10
print(perc, len(df_stats[(df_stats['pre_covid'] >= np.percentile(df_stats['pre_covid'].values,perc)) & 
         (df_stats['during_covid'] >= np.percentile(df_stats['during_covid'].values,perc)) & 
         (df_stats['post_covid'] >= np.percentile(df_stats['post_covid'].values,perc))]))

perc = 20
print(perc, len(df_stats[(df_stats['pre_covid'] >= np.percentile(df_stats['pre_covid'].values,perc)) & 
         (df_stats['during_covid'] >= np.percentile(df_stats['during_covid'].values,perc)) & 
         (df_stats['post_covid'] >= np.percentile(df_stats['post_covid'].values,perc))]))

perc = 25
print(perc, len(df_stats[(df_stats['pre_covid'] >= np.percentile(df_stats['pre_covid'].values,perc)) & 
         (df_stats['during_covid'] >= np.percentile(df_stats['during_covid'].values,perc)) & 
         (df_stats['post_covid'] >= np.percentile(df_stats['post_covid'].values,perc))]))

In [ ]:
df_stats.describe()

In [ ]:
# tweets de los usuarios que tienen en todos los períodos
no_covid_2019 = pd.to_datetime(no_covid_2019)
no_covid = pd.to_datetime(no_covid)
pre_covid = pd.to_datetime(pre_covid)
post_covid = pd.to_datetime(post_covid)

df_tweets_db = df_tweets_db[df_tweets_db['created_at'] >= pd.to_datetime(no_covid_2019)]

df_tweets_db = df_tweets_db[df_tweets_db['user_id'].isin(df_stats.index)]
df_tweets_db_no = df_tweets_db[df_tweets_db['created_at'] <= no_covid]
df_tweets_db_pre = df_tweets_db[(df_tweets_db['created_at'] > no_covid) & (df_tweets_db['created_at'] <= pre_covid)]
df_tweets_db_during = df_tweets_db[(df_tweets_db['created_at'] > pre_covid) & (df_tweets_db['created_at'] < post_covid)]
df_tweets_db_post = df_tweets_db[df_tweets_db['created_at'] >= post_covid]

len(df_tweets_db),len(df_tweets_db_no),len(df_tweets_db_pre),len(df_tweets_db_during),len(df_tweets_db_post)

In [ ]:
df_tweets_db.describe()
# df_tweets_db_no.describe()
# df_tweets_db_pre.describe()
# df_tweets_db_during.describe()
# df_tweets_db_post.describe()

In [ ]:
df_tweets_db.groupby(by='user_id')[['user_id']].count().describe()
# df_tweets_db_no.groupby(by='user_id')[['user_id']].count().describe()
# df_tweets_db_pre.groupby(by='user_id')[['user_id']].count().describe()
# df_tweets_db_during.groupby(by='user_id')[['user_id']].count().describe()
# df_tweets_db_post.groupby(by='user_id')[['user_id']].count().describe()
# get_stats(df_tweets_db,no_covid,pre_covid,post_covid).describe()

In [ ]:
import plotly.express as px

fig = px.scatter(df_stats.reset_index(), x="pre_covid", y="post_covid", size="during_covid", size_max=30)
fig.show()

### 

In [ ]:
# tweets originales only
df_original_tweets = df_tweets_db[(df_tweets_db['replied_id'] == -1) & (df_tweets_db['retweeted_id'] == -1) & (df_tweets_db['quoted_id'] == -1)]
df_original_tweets = df_original_tweets[df_original_tweets['user_id'].isin(set(df_stats.index))]
# df_stats_originales = get_stats(df_original_tweets,no_covid,pre_covid,post_covid)
# df_stats_originales.describe()

In [ ]:
df_original_tweets = df_tweets_db[(df_tweets_db['retweeted_id'] == -1)]
df_original_tweets = df_original_tweets[df_original_tweets['user_id'].isin(set(df_stats.index))]

df_stats_originales = get_stats(df_original_tweets,no_covid,pre_covid,post_covid)
df_stats_originales.describe()

In [ ]:
df_original_tweets
df_original_tweets.groupby(by='user_id')[['user_id']].count().describe()

In [ ]:
df_original_tweets = df_original_tweets[df_original_tweets['user_id'].isin(df_stats.index)]
df_original_tweets_no = df_original_tweets[df_original_tweets['created_at'] <= no_covid]
df_original_tweets_pre = df_original_tweets[(df_original_tweets['created_at'] > no_covid) & (df_original_tweets['created_at'] <= pre_covid)]
df_original_tweets_post = df_original_tweets[df_original_tweets['created_at'] >= post_covid]
df_original_tweets_during = df_original_tweets[(df_original_tweets['created_at'] > pre_covid) & (df_original_tweets['created_at'] < post_covid)]

In [ ]:
df_original_tweets_no

In [ ]:
def get_engagement(df_original_tweets):
    print('reply',100 * len(df_original_tweets[df_original_tweets['reply_count'] > 0]) / len(df_original_tweets))
    print('like',100 * len(df_original_tweets[df_original_tweets['favorite_count'] > 0]) / len(df_original_tweets))
    print('RT',100 * len(df_original_tweets[df_original_tweets['retweet_count'] > 0]) / len(df_original_tweets))
    print('any',100 * len(df_original_tweets[(df_original_tweets['reply_count'] > 0) | (df_original_tweets['favorite_count'] > 0) | (df_original_tweets['retweet_count'] > 0)]) / len(df_original_tweets))
    print('---------')

In [ ]:
get_engagement(df_original_tweets)
get_engagement(df_original_tweets_no)
get_engagement(df_original_tweets_pre)
get_engagement(df_original_tweets_during)
get_engagement(df_original_tweets_post)

In [ ]:
df_original_tweets.describe()
# df_original_tweets_no.describe()
# df_original_tweets_pre.describe()
# df_original_tweets_during.describe()
# df_original_tweets_post.describe()

In [ ]:
df_original_tweets[['favorite_count','retweet_count','reply_count']].corr()

In [ ]:
col = 'media_count'
col = 'mentions_count'
col = 'url_count'
col = 'hashtag_count'
print(len(df_original_tweets[df_original_tweets[col] > 0]))
print(len(df_original_tweets_no[df_original_tweets_no[col] > 0]))
print(len(df_original_tweets_pre[df_original_tweets_pre[col] > 0]))
print(len(df_original_tweets_during[df_original_tweets_during[col] > 0]))
print(len(df_original_tweets_post[df_original_tweets_post[col] > 0]))

In [ ]:
df_not_original_tweets = df_tweets_db[(df_tweets_db['retweeted_id'] != -1)]
df_not_original_tweets = df_not_original_tweets[df_not_original_tweets['user_id'].isin(set(df_stats.index))]
df_stats_not_originales = get_stats(df_not_original_tweets,no_covid,pre_covid,post_covid)
df_stats_not_originales

In [ ]:
df_not_original_tweets.groupby(by='user_id')[['user_id']].count().describe()

In [ ]:
print(len(df_not_original_tweets))
print(len(df_not_original_tweets[df_not_original_tweets['created_at'] <= no_covid]))
print(len(df_not_original_tweets[(df_not_original_tweets['created_at'] > no_covid) & (df_not_original_tweets['created_at'] <= pre_covid)]))
print(len(df_not_original_tweets[(df_not_original_tweets['created_at'] > pre_covid) & (df_not_original_tweets['created_at'] < post_covid)]))
print(len(df_not_original_tweets[df_not_original_tweets['created_at'] >= post_covid]))

df_stats_not_originales.describe()

In [ ]:
df_not_original_tweets = df_tweets_db[(df_tweets_db['replied_id'] != -1) | (df_tweets_db['retweeted_id'] != -1) | (df_tweets_db['quoted_id'] != -1)]
df_not_original_tweets = df_not_original_tweets[df_not_original_tweets['user_id'].isin(set(df_stats.index))]
df_stats_not_originales = get_stats(df_not_original_tweets,no_covid,pre_covid,post_covid)
df_stats_not_originales

In [ ]:
df_replies_tweets = df_tweets_db[(df_tweets_db['replied_id'] != -1)]
df_replies_tweets = df_replies_tweets[df_replies_tweets['user_id'].isin(set(df_stats.index))]
df_replies_tweets = get_stats(df_replies_tweets,pre_covid,post_covid)
df_replies_tweets

In [ ]:
df_replies_tweets = df_tweets_db[(df_tweets_db['retweeted_id'] != -1)]
df_replies_tweets = df_replies_tweets[df_replies_tweets['user_id'].isin(set(df_stats.index))]
df_replies_tweets = get_stats(df_replies_tweets,pre_covid,post_covid)
df_replies_tweets

In [ ]:
df_replies_tweets = df_tweets_db[(df_tweets_db['quoted_id'] != -1)]
df_replies_tweets = df_replies_tweets[df_replies_tweets['user_id'].isin(set(df_stats.index))]
df_replies_tweets = get_stats(df_replies_tweets,pre_covid,post_covid)
df_replies_tweets

In [ ]:
# Esto está para todos los tweets, se puede filtrar de acuerdo a lo que se desee, yo haría RT vs el resto.
def get_subset_tweets(df, which_tweets):
    if which_tweets == 'all':
        df_tweets_selected = df
    elif which_tweets == 'replies':
        df_tweets_selected = df[df['replied_id'] != -1]
    elif which_tweets == 'retweets':
        df_tweets_selected = df[df['retweeted_id'] != -1]
    elif which_tweets == 'quotes':
        df_tweets_selected = df[df['quoted_id'] != -1]
    elif which_tweets == 'no-rt': # todo lo que no sean RTs
        df_tweets_selected = df[df['retweeted_id'] == -1]
    return df_tweets_selected

In [ ]:
which_tweets = 'replies'
df_tweets_selected = get_subset_tweets(df_tweets_db, which_tweets)
len(df_tweets_selected)

In [ ]:
column = 'media_count'
agrr = 'count'
get_stats(df_original_tweets[df_original_tweets['media_count'] > 0].reset_index(),pre_covid,post_covid,column='tweet_id', agrr=agrr).describe()

#### Analizar los snapshots para obtener las cantidades de followees/followers por período

Teniendo en cuenta los snapshots que bajé, ver cuál es el snapshot más cercano a las fechas de corte para estimar los promedios en cada bloque.

OJO!! En algunos de los snapshots, el valor quedó pegado a algún otro, con lo cual, hay que revisarlos manualmente.

In [ ]:
df_snapshots_stats = pd.read_pickle('C:/Users/Anto/Dropbox/00_papers/df_stats.pickle')
df_snapshots_stats = df_snapshots_stats.merge(df_users_db.reset_index()[['screen_name','user_id']],left_on='screenname',right_on='screen_name')
df_snapshots_stats = df_snapshots_stats[df_snapshots_stats['user_id'].isin(df_tweets_db['user_id'])]
df_snapshots_stats['timestamp'] = [pd.to_datetime(datetime.strptime(x, '%Y%m%d%H%M%S')).to_pydatetime() if type(x) is str else None for x in df_snapshots_stats['timestamp']]
df_snapshots_stats

In [ ]:
from datetime import timedelta

df_pre = df_snapshots_stats[(df_snapshots_stats['timestamp'] <= pre_covid + timedelta(days=1)) & (df_snapshots_stats['timestamp'] >= pre_covid - timedelta(days=1))]
df_pre = df_pre.sort_values(by='timestamp').drop_duplicates('screenname',keep='last')
df_pre

In [ ]:
df_post = df_snapshots_stats[(df_snapshots_stats['timestamp'] <= post_covid + timedelta(days=3)) & (df_snapshots_stats['timestamp'] >= post_covid - timedelta(days=3))]
df_post = df_post.sort_values(by='timestamp').drop_duplicates('screenname',keep='last')
df_post

#### Obtener valor más actual para las stats de los usuarios

Como los datos de los usuarios están desactualizados en la db, obtener los valores a día de hoy.
Alcanza con hacer una query a la API por fuera.

In [ ]:
import random
import re
import requests

def get_bearer_token():
    url = "https://abs.twimg.com/responsive-web/client-web/main.90f9e505.js"
    user_agent = f'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.{random.randint(0, 9999)} Safari/537.{random.randint(0, 99)}'
    
    headers = {'user-agent': user_agent}
    response = requests.get(url,headers=headers)
    regex = re.compile("s=\"AAAAA[^\"]+\"")

    matches = regex.findall(response.text)[0]
    bearer = matches[3:-1]

    url = 'https://twitter.com/i/user/0'

    r = requests.get(url, headers = {'User-Agent': user_agent})
    match = re.search(r'document\.cookie = decodeURIComponent\("gt=(\d+); Max-Age=10800; Domain=\.twitter\.com; Path=/; Secure"\);', r.text)
    if match is not None:
        token = match.group(1)
    elif 'gt' in r.cookies:
        token = r.cookies['gt']
    else:
        r = requests.post('https://api.twitter.com/1.1/guest/activate.json', data = b'', headers = headers)
        o = r.json()
        token = o.get('guest_token')
    
    return bearer, token

bearer, guest_token = get_bearer_token()
bearer, guest_token

In [ ]:
from bs4 import BeautifulSoup
import unicodedata
from collections import deque
from tqdm.notebook import tqdm
import pandas as pd
import time

def get_page(search_url,headers=None,params=None): # para poder controlar los problemas -- AGREGAR AL OTRO CODIGO!
    
    timeout = True
    retries = 10
    response = None

    while retries > 0 and timeout:
        retries -= 1
        try: 
            if headers is not None and params is not None:
                rr = requests.get(search_url, headers=headers, params=params,timeout=10)
            elif headers is not None:
                rr = requests.get(search_url, headers=headers,timeout=10)
            elif params is not None:
                rr = requests.get(search_url, params=params,timeout=10)
            else:
                rr = requests.get(search_url,timeout=10)

            if rr.status_code == 200:
                timeout = False
                response = rr
            else:
                print('Temporarily Unavailable ...',rr.status_code)
                if rr.status_code == 403 or rr.status_code == 429 or rr.status_code == 401:
                    bearer, guest_token = get_bearer_token()
                    time.sleep(3)
        except requests.exceptions.RequestException as e:
            timeout = True
            print('Temporarily Unavailable ...')
            time.sleep(3)
    return response

import urllib.parse
import json

def get_user_current_info(user_id): 
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46',
              }
    response = get_page('https://twitter.com/intent/user?user_id='+str(user_id),headers=headers)
    
    if response is None:
        return None
        
    headers = {}
    headers['user-agent'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.46'
    headers['x-guest-token'] = guest_token
    headers['authorization'] = 'Bearer '+bearer

    url = 'https://twitter.com/i/api/graphql/I5nvpI91ljifos1Y3Lltyg/UserByRestId'
    params = {'userId': str(user_id), 'withSafetyModeUserFields': True, 'withSuperFollowsUserFields': True}
    params = urllib.parse.urlencode({'variables': json.dumps(params, separators = (',', ':'))}, quote_via = urllib.parse.quote)
    response = get_page(url,headers=headers,params=params)
    
    if response is None:
        return None
    
    response = response.json()['data']['user']

    if 'result' not in response:
        return None
    
    if response['result']['__typename'] == 'UserUnavailable':
        return None
    
    return {'user_id' : user_id,
            'screename':response['result']['legacy']['screen_name'],
            'followers':response['result']['legacy']['followers_count'], 
            'followees':response['result']['legacy']['friends_count'], 
            'listed':response['result']['legacy']['listed_count'],
            'tweets':response['result']['legacy']['statuses_count']} 

In [ ]:
listi = deque()
for x in tqdm(df_users_db.index):
    info = get_user_current_info(x)
    if info is not None:
        listi.append(info)
        
df_user_current_stats = pd.DataFrame(listi)
df_user_current_stats.to_pickle('df_user_current_stats.pickle')
df_user_current_stats

In [ ]:
df_user_current_stats = pd.read_pickle('df_user_current_stats.pickle')
# df_user_current_stats[df_user_current_stats['user_id'].isin(df_stats.index)].describe()
df_user_current_stats.sort_values(by='followers')